## Carbon Monitoring Project

In [ ]:
import holoviews as hv
import pandas as pd
hv.extension('bokeh')

This notebook aims to visualize the data used in the carbon monitoring project [nee_data_fusion](https://github.com/greyNearing/nee_data_fusion/) using Python tools. The goal is to establish how the relevant ML workflow can be expressed in Python.

To run this notebook, you will need to symlink the `data` directory of the [nee_data_fusion](https://github.com/greyNearing/nee_data_fusion/) to `flux_data` in the `examples` directory of `EarthML`. In addition, you will need `RSIF_2007_2016_05N_01L.mat` in the `examples` directory which you can download from https://gentinelab.eee.columbia.edu/content/datasets

### Loading FluxNet data ``extract_fluxnet.m``

[FluxNet](http://fluxnet.fluxdata.org/) is a worldwide collection of sensor stations that record a number of local variables relating to atmospheric conditions, solar flux and soil moisture. The data is the [nee_data_fusion](https://github.com/greyNearing/nee_data_fusion/) repository is expressed as a collection of CSV files where the site names are expressed in the filenames.

In [ ]:
# %read in FluxNet CSVs and replace NaNs with zeros
import numpy as np
import datetime
import os
sites = [fname.split('_')[1] for fname in os.listdir('./flux_data/dailies/')]

latlon = pd.read_csv('./flux_data/latlon.csv', header=None, names=['site', 'lat', 'lon'])

def site_lat_lon(latlon, site):
    location = latlon[latlon['site'] == site]
    if 0 in [len(location['lat']), len(location['lat'])]:
        return None, None
    return float(location['lat']), float(location['lon'])

def _parse_days(integer):
    """ `integer` date as `20180704` to represent July 4th, 2018"""
    x = str(integer)
    d = {'year': int(x[:4]), 'month': int(x[4:6]), 'day': int(x[6:])}
    day_of_year = datetime.datetime(d['year'], d['month'], d['day']).timetuple().tm_yday
    return day_of_year

def clean(df, timestamp_col="TIMESTAMP", keep=[], drop=[], predict=''):
    limit = -9990
    for i in range(50):
        df = df.replace(limit - i, np.nan)
        
    i = [col in df.columns for col in drop]
    df.drop(columns=drop, inplace=True)
    i = [col in df.columns for col in drop]
    
    df = df.fillna(0)
    df['DOY'] = df['TIMESTAMP'].apply(_parse_days)  
    df.pop('TIMESTAMP')
    X = df[keep]
    y = df[predict]
    return X, y

def load_fluxnet_site(site):
    # dataRaw(dataRaw <= -9990) = 0/0 (is NaN?)
    # NaN -> zero
    prefix = 'FLX_{site}_FLUXNET'.format(site=site)
    filenames = [fname for fname in os.listdir('./flux_data/dailies/')
                if fname.startswith(prefix)]
    assert len(filenames) == 1
    filename = filenames[0]
    
    raw_daily = pd.read_csv('./flux_data/dailies/{filename}'.format(filename=filename))    
    
    keep =  ['P_ERA',
             'TA_ERA',
             'PA_ERA',
             'SW_IN_ERA',
             'LW_IN_ERA',
             'WS_ERA',
             'SWC_F_MDS_1', #'SWC_F_MDS_2', 'SWC_F_MDS_3',
             'TS_F_MDS_1', #'TS_F_MDS_2', 'TS_F_MDS_3',
             'VPD_ERA',
             'DOY']
    drop = ["GPP_DT_VUT_USTAR50",
            "GPP_DT_CUT_USTAR50",
            "LE_F_MDS",
            "H_F_MDS"]
    predict = ["NEE_CUT_USTAR50",
               "NEE_VUT_USTAR50"]
    X, y = clean(raw_daily, keep=keep, drop=drop, predict=predict[0])
    return X, y

In [ ]:
X, y = load_fluxnet_site(sites[0])
print('Example FLUXNET data for site {site}:'.format(site=sites[0]))
print('Observations: {}, variables per observation: {}'.format(*X.shape))
X.describe()


Many of the FluxNet variables are expressed relative to a historical reference called ERA: 

* ERA-15: December 1978 to February 1994.
* ERA-40: Reanalysis from September 1957 through August 2002
  
Relevant links: [ERA](https://en.wikipedia.org/wiki/ECMWF_re-analysis) and [NCAR_Reanalysis](https://en.wikipedia.org/wiki/NCEP/NCAR_Reanalysis).

 
The goal in the matlab code is to build a super-array ``allData`` that spans 8766 Days x 22 Variables and 209 sites. Note that the 8766 span is derived from the *FluxNet* data not ERA. In the repository, I see 210 files with 21 variables. Some of the key variables:

* TIMESTAMP (Parsed out): YYYYMMDDHHMM	ISO timestamp – short format
* YEAR: Self explanatory
* DOY: Day of year.

Here are some of the relevant FluxNet variables recorded at each site. The definitions may be found [here](http://fluxnet.fluxdata.org/data/fluxnet2015-dataset/fullset-data-product/):
   
* P_ERA : Precipitation, downscaled from ERA, linearly regressed using measured only site data
* TA_ERA: Air temperature, downscaled from ERA, linearly regressed using measured only site data
* PA_ERA: Atmospheric pressure, downscaled from ERA, linearly regressed using measured only site data
* SW_IN_ERA : Shortwave radiation, incoming, downscaled from ERA, linearly regressed using measured only site data (negative values set to zero)
* LW_IN_ERA : Longwave radiation, incoming, downscaled from ERA, linearly regressed using measured only site data
* WS_ERA: Wind speed, downscaled from ERA, linearly regressed using measured only site data
* SWC_F_MDS_1/2/3: Soil water content, gapfilled with MDS (numeric index “#” increases with the depth, 1 is shallowest)
* TS_F_MDS_1/2/3: Soil temperature, gapfilled with MDS (numeric index “#” increases with the depth, 1 is shallowest)
* VPD_ERA:  Vapor Pressure Deficit, downscaled from ERA, linearly regressed using measured only site data

We'll use the above variables to predict:

* NEE_CUT_USTAR50 : Net Ecosystem Exchange, using Constant Ustar Threshold (CUT) across years, from 50 percentile of USTAR threshold
* NEE_VUT_USTAR50: Net Ecosystem Exchange, using Variable Ustar Threshold (VUT) for each year, from 50 percentile of USTAR threshold

Other variables should not be included as features, either because they are derived from NEE:

* GPP_DT_VUT_USTAR50: Gross Primary Production, from Daytime partitioning method, based on NEE_VUT_USTAR50
* GPP_DT_CUT_USTAR50: Gross Primary Production, from Daytime partitioning method, based on NEE_CUT_USTAR50

or because there is no suitable proxy for them in the satellite data:

* LE_F_MDS: Latent heat flux, gapfilled using MDS method
* H_F_MDS: Sensible heat flux, gapfilled using MDS method


Here, we will focus on predicting ``NEE_CUT_USTAR50``. 

Hypothesis : RSIF is more about vegetation than solar flux. Idea is that RSIF reflects carbon capture by vegetation.

NEE: Soil flux and photosynthesis of carbon.
Note: Not trying to separate these contributions.
Goal: Finding features. Adding features.

### Adding RSIF data ``collocate_data_types.m``

RSIF is the 'Reconstructed Solar Induced Fluorescence' expressing solar energy flux (power) per meter squared arriving on the Earth's surface derived from a vegetation signal.

In [ ]:
import scipy.io
rsif = scipy.io.loadmat('./flux_data/RSIF_2007_2016_05N_01L.mat')


The goal now is to add the RSIF time series added at positions of stations, then normalize the time dimension (resample to make the data sets have the same temporal sampling) and finally to perform a linear regression analysis on the combined data.

Note that the matlab file reference above was downloaded from https://gentinelab.eee.columbia.edu/content/datasets

In [ ]:
%%opts Image [width=700 height=500 clipping_colors={'NaN': 'gray'}] Points (marker='x' color='cyan')

mdata = rsif['RSIF'] # NaNs outside of land area.
def rsif_image(day):
    return hv.Image(mdata[:,:,day], kdims=['lon', 'lat'], vdims=['RSIF'], bounds=(-180,-90,180,90))

rsif_dmap = hv.DynamicMap(rsif_image, kdims=['day']).redim.values(day=range(mdata.shape[2]))
raw_site_positions = {site:site_lat_lon(latlon, site) for site in sites}
site_positions = {site:(lon,lat) for (site, (lat,lon)) in raw_site_positions.items() if None not in (lat,lon)}
rsif_dmap * hv.Points(site_positions.values())

* Algorithm might be useful: Smoothing?
* Mismatched temporospatial satellite imagery.

1. Timestamp per lat/lon. Comes with triples. 
2. Polar: e.g 1pm local time.
3. Global (processed product?)

https://science.nasa.gov/earth-science/earth-science-data/data-processing-levels-for-eosdis-data-products

1. Level 0: Raw data (direct sensor flux). Highest resolution. Minimal model - good for ML.
2. Level 1: Sensor geometry.
3. Level 2: spatial regridding  
4. Level 3: check: physical variable. Primary science product.
5. Level 4: check: model added value.



## Sampling the RSIF signal at the sites

In [ ]:
tables = []
for day in range(mdata.shape[2]):
    image = rsif_dmap.select(day=day)
    table = image.sample(samples=site_positions.values())
    tables.append(table.add_dimension('site', 0, site_positions.keys()))

In [ ]:
tabulated = hv.HoloMap({day:tables[day] for day in range(mdata.shape[2])}, kdims=['Day'])
tabulated

## Viewing the RSIF time series per site

In [ ]:
site_RSIF = tabulated.table().to(hv.Curve, 'Day', 'RSIF').drop_dimension(['lat', 'lon'])
site_RSIF

### biweekly_averaging.m

'RSIF' is put onto the same time sampling as the fluxnet data, adding an 'RSIF' field to the fluxnet dataframe for each site.

### main_lr_fluxnet.m


Finding the informative variables.

Main step:

```
model{s} = stepwiselm(X,y,''constant'',''Criterion'',''aic'',''Upper'',''linear'')
```

Using [stepwiselm](https://uk.mathworks.com/help/stats/stepwiselm.html):

```
mdl = stepwiselm(X,y,modelspec) creates a linear model of the responses y to the predictor variables in the data matrix X, using stepwise regression to add or remove predictors. modelspec is the starting model for the stepwise procedure.
```

Where ``allData`` is the table (effectively the dataframe):

```
cols= [3:11,14:22];
X = allData(:,cols,s);
y = allData(:,12,s);
```

* 3:11 : 'LAT','LON','P_ERA','TA_ERA','PA_ERA','SW_IN_ERA','LW_IN_ERA','WS_ERA','LE_F_MDS'
* 12: 'H_F_MDS' (Sensible heat flux, gapfilled using MDS method)

Check. NEE is the desired predicted. Column 13.

### main_ann_fluxnet.m

Main step for size ``s`` in ``trainANN`` using [``train``](https://uk.mathworks.com/help/nnet/ref/train.html) in the neural network toolbox:

```
% net = fitnet(parms.nodes,parms.trainFcn);
net = feedforwardnet(parms.nodes,parms.trainFcn);
[net,~] = train(net,x,t);
```

```
Xdex= [2:10,15,18,21,24];
Ydex = 13;
```

